In [11]:
import pandas as pd
import reverse_geocode

def transform_metadata(df):
    df.drop(columns=['description', 'hours', 'MISC', 'relative_results', 'url', 'price'], inplace=True)
    df.dropna(subset=['category'], inplace=True)

    palabras_claves = ['restaurant', 'food', 'steakhouse', 'mexican', 'pizzeria', 'american', 'asian']

    def filter_list_by_keywords(lst, palabras_claves):
        return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]

    df['category'] = df['category'].apply(lambda x: filter_list_by_keywords(x, palabras_claves))
    df['category'] = df['category'].fillna('')
    df = df[df['category'].apply(lambda x: len(x) > 0)]

    df.reset_index(drop=True, inplace=True)
    df = df[df['state'] != 'Permanently closed']

    df.drop_duplicates(subset='gmap_id', inplace=True)

    df['city'] = df['address'].str.extract(r',\s*([^,]+),\s*[A-Z]{2}\s+\d{5}', expand=False)
    df['zip_code'] = df['address'].str.extract(r'(\d{5})$', expand=False)

    def get_location(row):
        coordinates = [(row['latitude'], row['longitude'])]
        location = reverse_geocode.search(coordinates)[0]
        if location.get('country_code') == 'US' and 'city' in location and 'state' in location:
            return pd.Series([location['city'], location['state']])
        else:
            return pd.Series([None, None])

    results = df.apply(get_location, axis=1, result_type='expand')
    df['city'] = results[0]
    df['state'] = results[1]

    df.fillna('Sin Datos', inplace=True)
    df.rename(columns={'estado': 'state'}, inplace=True)

    df = df[['gmap_id', 'name', 'address', 'city', 'state', 'zip_code', 'latitude', 'longitude', 'category']]

    def abreviacion_estado(row):
        estados = {'New York': 'NY', 'California': 'CA', 'Texas': 'TX', 'Florida': 'FL', 'Pennsylvania': 'PA'}
        return estados.get(row['state'], 'Sin Datos')

    df['state'] = df.apply(abreviacion_estado, axis=1)
    df = df[df['state'].isin(['FL', 'TX', 'CA', 'NY', 'PA'])]

    return df

0                        Koreatown
1                        La Mirada
3                        Pennsport
4                            Lā‘ie
5        Lochmoor Waterway Estates
                   ...            
12275                    Whitehall
12277                    Whitehall
12278                      Camilla
12279                        Cairo
12280                    Rochester
Name: 0, Length: 7825, dtype: object

In [16]:
df=pd.read_json(r'E:\Data Science\PF\Datasets\Google Maps\metadata-sitios\1.json',lines=True)
df=transform_metadata(df)

5


In [17]:
df

,gmap_id,name,address,city,state,zip_code,latitude,longitude,category
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Koreatown,CA,90005,34.058092,-118.292130,[Korean restaurant]
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",La Mirada,CA,90638,33.916402,-118.010855,[Restaurant]
3,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",Pennsport,PA,19147,39.940293,-75.150923,[Restaurant]
4,0x7c00456eecad3111:0x8217f9600c51f33,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",Lā‘ie,Sin Datos,96762,21.637796,-157.920714,[Restaurant]
5,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",Lochmoor Waterway Estates,FL,33990,26.641377,-81.940545,[Restaurant]
...,...,...,...,...,...,...,...,...,...
12275,0x8838629ba011ece9:0x9b75b19079ad45e7,Subway,"Subway, 3657 E Main St, Whitehall, OH 43213",Whitehall,Sin Datos,43213,39.954151,-82.900727,"[Fast food restaurant, Takeout Restaurant, Res..."
12277,0x8838632a747705cd:0x15a209d12ff92a1e,taqueria la chaparrita #1,"taqueria la chaparrita #1, 3525 E Livingston A...",Whitehall,Sin Datos,43227,39.945708,-82.901851,[Food producer]
12278,0x88ed9c22ebc8e5bf:0xfea25da7fd6a2873,E-Z Access Restarunt And Carwash,"E-Z Access Restarunt And Carwash, 25 Twitty St...",Camilla,Sin Datos,31730,31.233745,-84.208377,[Chinese restaurant]
12279,0x88edadfb481204e9:0x866f04e454b7a8f6,Brother's Fishing Co and Country Market,"Brother's Fishing Co and Country Market, 2330 ...",Cairo,Sin Datos,39828,30.828153,-84.180208,[Seafood market]


In [13]:
def get_location(row):
    coordinates = [(row['latitude'], row['longitude'])]
    location = reverse_geocode.search(coordinates)[0]
    if location.get('country_code') == 'US' and 'city' in location and 'state' in location:
        return pd.Series([location['city'], location['state']])
    else:
        return pd.Series([None, None])

In [15]:
results = df.apply(get_location, axis=1, result_type='expand')
df['city'] = results.apply(lambda x: x['city'], axis=1)
df['state'] = results.apply(lambda x: x['state'], axis=1)

KeyError: 'city'

In [9]:
df

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state,city,zip_code
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4,18,California,Koreatown,90005
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,[Restaurant],4.5,18,California,La Mirada,90638
3,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,[Restaurant],4.0,8,Pennsylvania,Pennsport,19147
4,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,21.637796,-157.920714,[Restaurant],4.4,18,Hawaii,Lā‘ie,96762
5,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,1,Florida,Lochmoor Waterway Estates,33990
...,...,...,...,...,...,...,...,...,...,...,...
12275,Subway,"Subway, 3657 E Main St, Whitehall, OH 43213",0x8838629ba011ece9:0x9b75b19079ad45e7,39.954151,-82.900727,"[Fast food restaurant, Takeout Restaurant, Res...",3.7,68,Ohio,Whitehall,43213
12277,taqueria la chaparrita #1,"taqueria la chaparrita #1, 3525 E Livingston A...",0x8838632a747705cd:0x15a209d12ff92a1e,39.945708,-82.901851,[Food producer],5.0,13,Ohio,Whitehall,43227
12278,E-Z Access Restarunt And Carwash,"E-Z Access Restarunt And Carwash, 25 Twitty St...",0x88ed9c22ebc8e5bf:0xfea25da7fd6a2873,31.233745,-84.208377,[Chinese restaurant],4.7,3,Georgia,Camilla,31730
12279,Brother's Fishing Co and Country Market,"Brother's Fishing Co and Country Market, 2330 ...",0x88edadfb481204e9:0x866f04e454b7a8f6,30.828153,-84.180208,[Seafood market],4.8,38,Georgia,Cairo,39828
